In [1]:
from sentence_transformers import SentenceTransformer, models, util
import pandas as pd
import numpy as np
from tqdm.autonotebook import trange
import torch
import os
import json
import pickle
import time
import faiss

model_name = './output/bi-encoder-sup_hlf-rbtl3-2022-06-09'
dataset_path = './dureader-retrieval-test1/test1.json'

embedding_cache_path = 'embeddings-hfl-rbtl3.pkl'
embedding_size = 1024    #Size of embeddings
top_k_hits = 50         #Output k hits

In [5]:
model = SentenceTransformer(model_name)

In [6]:
index = faiss.read_index('faiss.index')

In [20]:
question_embeddings = model.encode(list(questions.question.values))
question_embeddings = question_embeddings / np.linalg.norm(question_embeddings, axis=1)[:, None]

In [5]:
questions = pd.read_json(dataset_path, lines=True)

In [36]:
torch.set_num_threads(12)
results_corpus_ids = []
for question_index in trange(0, len(question_embeddings), 10): 
    question_embedding = question_embeddings[question_index:question_index+10]
#     question_embedding = np.expand_dims(question_embedding, axis=0)
    distances, corpus_ids = index.search(question_embedding, top_k_hits)
    results_corpus_ids.extend(corpus_ids)

  0%|          | 0/2500 [00:00<?, ?it/s]

In [44]:
results_id = {}
for _index, corpus_ids in zip(range(0, len(question_embeddings)), results_corpus_ids):
    results_id[_index] = list(corpus_ids)
    
with open('results_id.pkl', "wb") as fOut:
    pickle.dump(results_id, fOut)

In [47]:
with open('./dureader/passage-collection/passage2id.map.json', 'r', encoding='utf-8') as f:
    passage2id_map = json.load(f)

In [55]:
results = {}
for key, value in results_id.items():
    question_id = questions.question_id[key]
    corpus_ids = [passage2id_map[str(i)] for i in value]
    results[question_id] = corpus_ids

In [56]:
with open('recall_results.json', 'w', encoding='utf-8') as f:
    json.dump(results, f, ensure_ascii=False, indent='\t')

In [2]:
print("Load pre-computed embeddings from disc")
with open(embedding_cache_path, "rb") as fIn:
    cache_data = pickle.load(fIn)
    corpus_sentences = cache_data['sentences']
    corpus_embeddings = cache_data['embeddings']

Load pre-computed embeddings from disc


In [3]:
with open('results_id.pkl', "rb") as f:
    results_id = pickle.load(f)

In [6]:
results_for_cross = {}
for key, value in results_id.items():
    question = questions.question[key]
    corpus = [corpus_sentences[i] for i in value]
    results_for_cross[question] = corpus

In [7]:
with open('results_for_cross.pkl', "wb") as fOut:
    pickle.dump(results_for_cross, fOut)

In [ ]:
# # We extract corpus ids and scores for the first query
# hits = [{'corpus_id': id, 'score': score} for id, score in zip(corpus_ids[0], distances[0])]
# hits = sorted(hits, key=lambda x: x['score'], reverse=True)

# print("Input question:", inp_question)
# for hit in hits[0:top_k_hits]:
#     print("\t{:.3f}\t{}".format(hit['score'], corpus_sentences[hit['corpus_id']]))

# # Approximate Nearest Neighbor (ANN) is not exact, it might miss entries with high cosine similarity
# # Here, we compute the recall of ANN compared to the exact results
# correct_hits = util.semantic_search(question_embedding, corpus_embeddings, top_k=top_k_hits)[0]
# correct_hits_ids = set([hit['corpus_id'] for hit in correct_hits])

# ann_corpus_ids = set([hit['corpus_id'] for hit in hits])
# if len(ann_corpus_ids) != len(correct_hits_ids):
#     print("Approximate Nearest Neighbor returned a different number of results than expected")

# recall = len(ann_corpus_ids.intersection(correct_hits_ids)) / len(correct_hits_ids)
# print("\nApproximate Nearest Neighbor Recall@{}: {:.2f}".format(top_k_hits, recall * 100))

# if recall < 1:
#     print("Missing results:")
#     for hit in correct_hits[0:top_k_hits]:
#         if hit['corpus_id'] not in ann_corpus_ids:
#             print("\t{:.3f}\t{}".format(hit['score'], corpus_sentences[hit['corpus_id']]))
# print("\n\n========\n")

In [ ]:
# train_samples = []
# with open('./dureader/train/cross.train.tsv', 'r', encoding='utf-8') as f:
#     for line in f.readlines():
#         data = line.rstrip().split('\t')
#         inp_example = [data[0][:256], data[2][:256], float(data[3])]
#         train_samples.append(inp_example)


# emb1 = model.encode('测试')
# emb2 = model.encode(inp_example[1])
# print('label: ', inp_example[2])
# print('sim: ', util.cos_sim(emb1, emb2))